<a href="https://colab.research.google.com/github/23maxb/RagImplementationWithGPT4o/blob/main/UI_for_Vectorized_Database_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Imports {"display-mode":"form"}
!pip install -qU \
  pinecone-client==3.1.0 \
  pinecone-datasets==0.7.0 \
  sentence-transformers==2.2.2 \
  pinecone-notebooks==0.1.1
!pip install openai
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.2/152.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.0/211.0 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.2/77.2 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 19.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 1.4.14 requires pydantic>=2.7.0, but you have pydantic 1.10.18 which is incompatible.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14

In [ ]:
# @title Setup {"vertical-output":true,"display-mode":"form"}
import time
from pinecone import Pinecone, ServerlessSpec
from google.colab import userdata
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device != 'cuda':
    print(f"You are using {device}. This is much slower than using "
          "a CUDA-enabled GPU. If on Colab you can change this by "
          "clicking Runtime > Change runtime type > GPU.")

model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

pc = Pinecone(api_key=(userdata.get('PINECONE_API_KEY') or 'PINECONE_API_KEY'))
cloud = 'aws'
region = 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)
index_name = 'semantic-search'


# check if index already exists (it shouldn't if this is first time)
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=model.get_sentence_embedding_dimension(),
        metric='cosine',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
# view index stats
index.describe_index_stats()

You are using cpu. This is much slower than using a CUDA-enabled GPU. If on Colab you can change this by clicking Runtime > Change runtime type > GPU.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1399}},
 'total_vector_count': 1399}

In [ ]:
# @title Update Context from Google Drive

import os
import PyPDF2
import glob
from tqdm.auto import tqdm
from google.colab import drive
drive.mount('/content/drive')

GoogleDriveLocation = "MattProject/PDFS/Added"#@param {type:"string"}
directory = "/content/drive/MyDrive/" + GoogleDriveLocation

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\n\n"
    return text

def process_directory(directory):
    # Get all PDF files in the directory
    pdf_files = glob.glob(os.path.join(directory, "*.pdf"))

    for pdf_file in pdf_files:
      if not os.path.isfile(pdf_file.replace(".pdf", "_extracted.txt")):
        print(f"Processing {pdf_file}...")

        # Extract text
        text = extract_text_from_pdf(pdf_file)

        # Save the extracted text
        output_file = os.path.splitext(pdf_file)[0] + "_extracted.txt"
        with open(output_file, "w", encoding="utf-8") as f:
            f.write(text)

        print(f"Extracted text saved to {output_file}")

# Usage
process_directory(directory)

pc = Pinecone(api_key=userdata.get('PINECONE_API_KEY'))

def get_txt_files_content(directory):
    result = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.txt'):
                file_path = os.path.join(root, file)
                with open(file_path, 'r') as f:
                    content = f.read()
                    split = content.split('.')
                    to_add = []
                    temp = ""
                    for s in split:
                      #500 length is arbitrary its just minimum size chunk of text to give
                      #change this number to increase the size of each block of text
                      if(len(temp) < 800):
                        temp += " " + s
                      else:
                        to_add.append(temp)
                        temp = ""
                    result.extend(to_add)
    return result

device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device != 'cuda':
    print(f"You are using {device}. This is much slower than using "
          "a CUDA-enabled GPU. If on Colab you can change this by "
          "clicking Runtime > Change runtime type > GPU.")

model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
model

pc.delete_index(index_name) #delete the index to create a new one
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=model.get_sentence_embedding_dimension(),
        metric='cosine',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
# view index stats
index.describe_index_stats()


batch_size = 128
vector_limit = 100000

txt_files_content = get_txt_files_content(directory)
noDuplicates = list(set(txt_files_content))
print(len(noDuplicates))
data = noDuplicates[:vector_limit]

for i in tqdm(range(0, len(data), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(data))
    # create IDs batch
    ids = [str(x) for x in range(i, i_end)]
    # create metadata batch
    metadatas = [{'text': text} for text in data[i:i_end]]
    # create embeddings
    xc = model.encode(data[i:i_end])
    # create records list for upsert
    records = zip(ids, xc, metadatas)
    # upsert to Pinecone
    index.upsert(vectors=records)

# check number of records in the index
index.describe_index_stats()


Mounted at /content/drive
You are using cpu. This is much slower than using a CUDA-enabled GPU. If on Colab you can change this by clicking Runtime > Change runtime type > GPU.
1399


  0%|          | 0/11 [00:00<?, ?it/s]

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1399}},
 'total_vector_count': 1399}

In [ ]:
# @title Prompt
from openai import OpenAI
from google.colab import userdata

query = "What are some of the biggest geopolitical risks we face in the next few years?" #@param {type:"string"}

# create the query vector
xq = model.encode(query).tolist()

# now query
xc = index.query(vector=xq, top_k=10, include_metadata=True)
for result in xc['matches']:
    #print(f"{round(result['score'], 2)}: {result['metadata']['text']}")
    query = f"{round(result['score'], 2)}: {result['metadata']['text']}" + query


client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": "You are a geoeconomic analyst that uses evidence in documents to answer questions. Using the given documents answer the question given.."},
    {"role": "user", "content": query}
  ]
)

res = completion.choices[0].message.content
print(res)

Based on the provided documents, some of the biggest geopolitical risks we face in the next few years include:

1. **Increased Protectionism and Trade Restrictions**:
   - Geopolitical rivalries have led to greater protectionism and a rise in the number of trade restrictions on national security grounds (Figure 2). The share of G20 imports affected by trade restrictive measures increased significantly over the past decade. 

2. **Geopolitical Uncertainty and Economic Implications**:
   - Geopolitical uncertainty, especially between major powers like the US and China, negatively impacts investment, stock prices, employment, and increases the probability of economic crises with larger downside risks to the global economy (Caldara and Iacoviello 2022).

3. **Fragmentation of Global Financial and Trade Systems**:
   - There is a growing risk of the world fragmenting into distinct economic blocs with different ideologies, political systems, technology standards, and trade systems. This frag